In [35]:
import gensim
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np
import re
import csv

import pandas as pd

import string
import nltk

In [36]:
pathToDatasets = '../datasets/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
word_vectors = api.load("glove-wiki-gigaword-100")


In [37]:
# positiveWordList = ['affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative']
# negativeWordList = ['varice', 'avaricious', 'avariciously', 'avenge', 'averse', 'aversion', 'aweful', 'awful', 'awfully', 'awfulness']

# dimensionsDictionary = {}
# countOfWords = 0;

# def countDimensions(listOfDimensions):
#   for dimension in listOfDimensions:
#     if (dimensionsDictionary.get(dimension) == None):
#       dimensionsDictionary[dimension] = 1;
#     else:
#       dimensionsDictionary[dimension] +=1;


In [38]:
# def sortedDictionary(dictUnsorted):
#   sorted_d = sorted(dictUnsorted.items(), key=lambda x: x[1], reverse=True)
#   return sorted_d;



In [39]:
# vocabulary = word_vectors.vocab;
# for posWord in positiveWordList:
#   if(posWord not in vocabulary):
#     continue  
#   posVector = word_vectors.word_vec(posWord)
#   for negWord in negativeWordList:
#     if(negWord not in vocabulary):
#       continue
#     negVector = word_vectors.word_vec(negWord)
#     diffBetween = posVector - negVector
#     sortedList = sorted(range(len(diffBetween)), key=lambda k: diffBetween[k], reverse=True)
#     similarityScore = word_vectors.similarity(posWord, negWord);
# #     print("Dimensions for word {0} and word {1} is {2} with similarity scores: {3}".format(posWord, negWord, sortedList[0:3], similarityScore))
#     countDimensions(sortedList[0:3]);
#     countOfWords+=1;
#     # print("----- {} -----".format(diffBetween));

# # # print(dimensionsDictionary);
# print(sortedDictionary(dimensionsDictionary));
# print("Count of number of word pairs : {0}".format(countOfWords))



In [40]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize



senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


In [41]:
def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=5)]
    return possibleReplacements

def cleanAndTokenizeText(text):
    text = text.lower()
    newString = ""
    for char in text:
        if char not in string.punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

In [51]:
def getAlternativeSentences(tweet, sentimentOfTweet):
    userInputTokens = cleanAndTokenizeText(tweet)
    print("Alternatives: ")
    for ind,word in enumerate(userInputTokens):
        score = senty.polarity_scores(word)['compound']
        newUserTokens = userInputTokens[:];
        if(score != 0.0 ):
            replacements = listReplacements(word)       
            if(replacements == []):
                return
            print("Word being changed : {0}".format(word));
            for newWord in replacements:
                newUserTokens[ind] = newWord;
                newString = ' '.join(newUserTokens)
                sentimentOfNewString = senty.polarity_scores(newString)['compound']
                if(sentimentOfNewString == 0):
                    continue
                if(abs(sentimentOfNewString) >= abs(sentimentOfTweet)):
                    print("{0} : {1} ++POS++".format(newString,sentimentOfNewString))
                else:
                    print("{0} : {1} --NEG--".format(newString,sentimentOfNewString))
            newUserTokens = userInputTokens[:];

In [56]:
def runThroughTweets():
    numberOfTweets = 50;
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=60, skiprows=30 )
    tweets = tweets_df.values

    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound'] # tweet[0] is string since [tweet]
        if(mainSentiment == 0):
            continue
        print("\n-NUM({2}) {0}:{1}-\n".format(tweet,mainSentiment, counter))
        getAlternativeSentences(tweet, mainSentiment)
    
runThroughTweets()


-NUM(0) some1 hacked my account on aim  now i have to make a new one:-0.4019-

Alternatives: 
Word being changed : hacked

-NUM(1)  i want to go to promote gear and groove but unfornately no ride there  i may b going to the one in anaheim in may though:-0.2144-

Alternatives: 
Word being changed : want
i do to go to promote gear and groove but unfornately no ride there i may b going to the one in anaheim in may though : -0.25 ++POS++
i n't to go to promote gear and groove but unfornately no ride there i may b going to the one in anaheim in may though : -0.25 ++POS++
i know to go to promote gear and groove but unfornately no ride there i may b going to the one in anaheim in may though : -0.25 ++POS++
i we to go to promote gear and groove but unfornately no ride there i may b going to the one in anaheim in may though : -0.25 ++POS++
i get to go to promote gear and groove but unfornately no ride there i may b going to the one in anaheim in may though : -0.25 ++POS++
Word being changed : 

Word being changed : ill
ill tell ya the story later not a good day and sick be workin for like three more hours : -0.7189 ++POS++
ill tell ya the story later not a good day and illness be workin for like three more hours : -0.6604 --NEG--
ill tell ya the story later not a good day and suffering be workin for like three more hours : -0.7009 ++POS++
ill tell ya the story later not a good day and hospitalized be workin for like three more hours : -0.4031 --NEG--
ill tell ya the story later not a good day and died be workin for like three more hours : -0.7435 ++POS++
Word being changed : like
ill tell ya the story later not a good day and ill be workin for even three more hours : -0.7909 ++POS++
ill tell ya the story later not a good day and ill be workin for well three more hours : -0.7101 ++POS++
ill tell ya the story later not a good day and ill be workin for you three more hours : -0.7909 ++POS++
ill tell ya the story later not a good day and ill be workin for such three more hours : 

Word being changed : yay
wahb im happy for you with your job but that also means less time for me and you : 0.3291 --NEG--
roon im happy for you with your job but that also means less time for me and you : 0.3291 --NEG--
vahl im happy for you with your job but that also means less time for me and you : 0.3291 --NEG--
kood im happy for you with your job but that also means less time for me and you : 0.3291 --NEG--
bruh im happy for you with your job but that also means less time for me and you : 0.3291 --NEG--
Word being changed : happy
yay im 'm for you with your job but that also means less time for me and you : 0.296 --NEG--
yay im feel for you with your job but that also means less time for me and you : 0.296 --NEG--
yay im 're for you with your job but that also means less time for me and you : 0.296 --NEG--
yay im i for you with your job but that also means less time for me and you : 0.296 --NEG--
yay im 'll for you with your job but that also means less time for me and you : 0.29

not sure what they are only that they are pos as much as i get to i dont think can trade away company assets sorry andy : -0.1436 --NEG--
Word being changed : assets
not sure what they are only that they are pos as much as i want to i dont think can trade away company investments sorry andy : -0.2411 ++POS++
not sure what they are only that they are pos as much as i want to i dont think can trade away company funds sorry andy : -0.2411 ++POS++
not sure what they are only that they are pos as much as i want to i dont think can trade away company asset sorry andy : 0.1376 --NEG--
not sure what they are only that they are pos as much as i want to i dont think can trade away company debt sorry andy : -0.5365 ++POS++
not sure what they are only that they are pos as much as i want to i dont think can trade away company investment sorry andy : -0.2411 ++POS++
Word being changed : sorry
not sure what they are only that they are pos as much as i want to i dont think can trade away company asset

Word being changed : lucky
gym attire today was puma singlet adidas shortsand black business socks and leather shoes fortunate did not run into any cute girls : 0.7096 ++POS++
gym attire today was puma singlet adidas shortsand black business socks and leather shoes happy did not run into any cute girls : 0.7717 ++POS++
gym attire today was puma singlet adidas shortsand black business socks and leather shoes unlucky did not run into any cute girls : 0.4588 --NEG--
gym attire today was puma singlet adidas shortsand black business socks and leather shoes 'd did not run into any cute girls : 0.4588 --NEG--
gym attire today was puma singlet adidas shortsand black business socks and leather shoes luck did not run into any cute girls : 0.7184 ++POS++
Word being changed : cute
gym attire today was puma singlet adidas shortsand black business socks and leather shoes lucky did not run into any adorable girls : 0.7184 ++POS++
gym attire today was puma singlet adidas shortsand black business socks

Word being changed : evil
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha enemies laugh : 0.0258 --NEG--
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha god laugh : 0.6597 ++POS++
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha demons laugh : 0.5106 ++POS++
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha beings laugh : 0.5106 ++POS++
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha wicked laugh : -0.0258 --NEG--
Word being changed : laugh
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha evil laughs : -0.3612 ++POS++
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha evil joke : -0.5423 ++POS++
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha evil chuckle : -0.4588 ++POS++
sorry babe my fam annoys me too thankfully theyre asleep right now muahaha evil cry : -0.8316 ++POS++
sorry b